In [ ]:
import csv
import re


In [ ]:
!gdown 1YmiXKfAks2KnuSfRNpgfVnvnWBcsFsQy


In [ ]:
!unzip ManchesterCorpus.zip

In [ ]:
import sys, os, re


def recurse_dirs(dir):
    outtext=[]
    outlist = []
    corpora = os.listdir(dir)
    for corpus in corpora:
        path = dir + corpus
        dirbin = []

        for d_path, d_names, f_names in os.walk(path):

            if any(fname.endswith('.cha') for fname in os.listdir(d_path)):

                # for f in_names:
                # if f.endswith('cha'):
                # print d_path + f
                subdir = d_path + "/"
                remove_embedded_returns(subdir)
                for f in os.listdir(subdir):
                    if f.endswith('.rem'):

                      outtext.append(get_file_text(subdir + f,f))
    return outtext

def get_file_text(file,filename):
    thistext = []
    previousline="START"
    with open(file, 'r') as infile:
        lines = infile.readlines()
        i = 0
        for line in lines:
             i=i+1

             if not re.match(".*xxx.*",line):
               if re.match("\*(MOT|INV|FAT)\:[^\.]+\.",line.rstrip('\n')):
                    speaker = re.search("\*(MOT|INV|FAT)\:",line)[0]
                    line = deal_with_variant_form_exceptions(line)
                    line = deal_with_variant_forms(line)
                    line = deal_with_marked_errors_etc(line)
                    # some residual rules
                    line = re.sub("\+\/+\.",".",line)
                    line = re.sub("\+\"\/? ","",line)
                    line = re.sub("\+\.\.\. ","",line)
                    line = re.sub("\+\/+\? ","",line)
                    line = re.sub("\+","",line)
                    line = remove_commas(line)
                    line = re.sub("\&-um ","",line)
                    line = re.sub("\&-er ","",line)
                    line = re.sub("\.\.\.",".",line)
                    # next rule needs consideration
                    # line = re.sub("([^ ]+)\@[a-z]","\\1",line)
                    line = re.sub("www","",line)

                    if i <= len(lines):
                      #and lines[i].strip().startswith('%mor:'):
                      response  = line.split(':',1)[1]
                      response = re.sub("\:","",response)
                      response_as_list=response.split()

                      if len(response_as_list) > 0 and "xxx" not in response_as_list:
                          thistext.append(" ".join(response_as_list))
    return(thistext)                  #print(file + " " + speaker + " : " + " ".join(response_as_list))
def remove_embedded_returns(my_dir):
    #global i
    import os
    import copy
    files = os.listdir(my_dir)
    for file in files:
        # print file
        if (('.cha' in file
             or '.cex' in file)
            and '.rem' not in file
            # and 'Helen.dd03' in file
                and '._' not in file):
            with open(my_dir + file, 'r') as infile:
                inlist = infile.readlines()
                # print len(inlist)
                for i in range(len(inlist)):
                    inlist[i] = inlist[i].split()
                    if inlist[i] == []:
                        inlist[i] = ['binthis']
                while ['binthis'] in inlist:
                    inlist.remove(['binthis'])
                # print len(inlist)
                for j in range(5):
                    for i in range(len(inlist)):
                        # print i, inlist[i]
                        if (len(inlist[i]) > 0
                            and inlist[i][0][0] in ['%', '*', '@', '%']
                            and i < len(inlist) - 2
                            and inlist[i+1][0][0] not in ['%', '*', '@', '%']
                                and inlist[i+1] != ['binthis']):
                            # print my_dir + file
                            # print 'old', inlist[i-1]
                            # print inlist[i]
                            # print inlist[i]
                            inlist[i] = copy.deepcopy(
                                inlist[i]) + copy.deepcopy(inlist[i+1])
                            inlist[i+1] = ['binthis']
                            # print 'new', inlist[i-1]
                            # print 'new1', inlist[i]
                        if inlist[i] == []:
                            inlist[i] = ['binthis']
                    # print len(inlist)
                    while ['binthis'] in inlist:
                        inlist.remove(['binthis'])
                    # print len(inlist)
                    # print 'done'
            with open(my_dir + file + '.rem', 'w') as outfile:
                for line in inlist:
                    outfile.write(' '.join(line) + '\n')


def deal_with_variant_form_exceptions(textline):

    textline = textline.replace('ahem ach [: I don’t]', 'I don’t')
    textline = textline.replace('Uh no [: don’t know]', 'don’t know')
    textline = textline.replace('goinga [: going to]', 'gonna')
    textline = textline.replace('gon [: going to]', 'gonna')
    textline = textline.replace('deres [: there is]', 'there’s')
    textline = textline.replace('don [: do not]', 'don’t')
    textline = textline.replace('hed+up [: fed up]', 'fed up')
    textline = textline.replace('hull+up [: full up]', 'full up')
    textline = textline.replace(
        'creabids [: creamy porridge]', 'creabids')

    return textline


def deal_with_variant_forms(textline):
    textline = textline.split(" ")
    newtextline = []
    lastitem = ''
    lastbutoneitem = ''
    #lastbuttwoitem = ''
    # print textline
    for item in textline:
        item = item.replace('(', '')
        item = item.replace(')', '')

        if item == '[:':
          break
        else:
          newtextline.append(item.lower())
    return " ".join(newtextline)

def deal_with_marked_errors_etc(textline):
    textline = textline.replace('\]\[', '] [')
    textline = re.sub('([^ ]),','\\1 ,', textline)
    textline = re.sub(',([^ ])',', \\1', textline)
    textline = re.sub('\<','', textline)
    textline = re.sub('\>','', textline)
    textline = re.sub('\[[^\[\]]*\] ?', '', textline)
    textline = re.sub('\(\.\) ?', '', textline)

    textline = textline.replace('0is ', '')
    return textline

def remove_commas(textline):
    outline = re.sub("\, ","",textline)
    return outline



In [ ]:
all_input = recurse_dirs("Manchester/")

In [ ]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
tagged_text = ""
with open as f:
with open('manchester_input_tagged.txt', 'w') as f:
    for i in range(len(all_input)):
       for utt in all_input[i]:
           doc = nlp(str(utt))
           f.write(" ".join([w.text + "_" + w.pos_ for w in doc]) + "\n")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!mv tagged_text /content/drive/